# Airfoil optimization

In [ ]:
from IPython.display import SVG, display

import torch
import torch.nn as nn
import torch.optim as optim
from torch import distributions
from torch.distributions import Normal


from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor, as_completed
from functools import partial

#stuff for neurofoil - pip install neuralfoil
import shutil
import numpy as np

import sys
import os
import contextlib
from collections import deque
import copy

import warnings
import pennylane as qml

import multiprocessing as mp
from multiprocessing import Pool, Lock, Value
import subprocess
import matplotlib.pyplot as plt
import subprocess

from scipy.optimize import differential_evolution
import uuid
import pandas as pd

print(os.getcwd())

In [ ]:
id_ = 1 
glock = mp.Lock()

def get_id(lock):
    global id_
    with lock:
        id_ += 1
    return id_

## Procedure 

Sources:
* http://servidor.demec.ufpr.br/CFD/bibliografia/aerodinamica/kulfan_2007.pdf
* https://www.tandfonline.com/doi/epdf/10.1080/19942060.2024.2445144
* https://github.com/peterdsharpe/NeuralFoil


Here the PINN is allready trained an provides the prediction. Therefore, the idea is, we shift the quantum opt. part to approximate a policy. 

In [ ]:
display(SVG(filename="optimprocedure.svg"))

In [ ]:
sys.path.append(os.path.abspath("../blade_design_tools"))
import create_airfoil as ca
airfoil_top_points, airfoil_bottom_points, camber_curve_points = ca.generate_shape(config_filepath="../blade_design_tools/airfoil_config.yaml")

In [ ]:
#modified shape_method - need controled file write!
def generate_shape(config_filepath="airfoil_config.yaml", config_dict=None, plot_foil=False):
    try:
        config = ca.load_config(config_filepath) if config_dict is None else config_dict
        
        # Handle optional input targets for chord and max thickness
        input_targets = config.get('input_targets', {})
        desired_chord = input_targets.get('desired_chord_length', None)
        desired_max_th = input_targets.get('desired_max_thickness', None)
        
        # Temporarily compute camber to get normalized chord (needed for adjustments)
        temp_camber_control_points, temp_camber_curve_points = ca.compute_curve_points(
            config['camber_line'], "camber", config['output_settings']['num_points_on_curve']
        )
        norm_le = temp_camber_curve_points[0]
        norm_te = temp_camber_curve_points[-1]
        norm_chord = np.linalg.norm(norm_te - norm_le)
        norm_x_span = norm_te[0] - norm_le[0]  # Should be 1.0
        
        # Adjust chord_length_for_export if desired_chord is set (to achieve desired geometric chord)
        if desired_chord is not None:
            config['output_settings']['chord_length_for_export'] = desired_chord * (norm_x_span / norm_chord)  # Equivalent to desired_chord * np.cos(np.deg2rad(np.abs(config['camber_line']['stagger_angle_deg'])))
        
        # 1. Compute Camber Line
        camber_control_points, camber_curve_points = ca.compute_curve_points(
            config['camber_line'], "camber", config['output_settings']['num_points_on_curve']
        )
       
        # 2. Compute Top Thickness Distribution
        top_thickness_control_points, top_thickness_curve_points = ca.compute_curve_points(
            config['top_thickness'], "top_thickness", config['output_settings']['num_points_on_curve']
        )
        # 3. Compute Bottom Thickness Distribution
        bottom_thickness_control_points, bottom_thickness_curve_points = ca.compute_curve_points(
            config['bottom_thickness'], "bottom_thickness", config['output_settings']['num_points_on_curve']
        )
        
        # Temporarily compute current normalized max thickness
        current_norm_thickness_along = top_thickness_curve_points[:, 1] + bottom_thickness_curve_points[:, 1]
        current_norm_max_th = np.max(current_norm_thickness_along)
        
        # Compute scale factor (reflects any chord adjustment)
        scale = config['output_settings']['chord_length_for_export'] / norm_x_span
        
        # Adjust thickness curves if desired_max_th is set
        if desired_max_th is not None:
            desired_norm_max_th = desired_max_th / scale
            th_scale_factor = desired_norm_max_th / current_norm_max_th
            top_thickness_curve_points[:, 1] *= th_scale_factor
            bottom_thickness_curve_points[:, 1] *= th_scale_factor
        
        # 4. Generate Full Airfoil Contour
        airfoil_top_points, airfoil_bottom_points = ca.generate_airfoil_contour(
            camber_curve_points,
            top_thickness_curve_points,
            bottom_thickness_curve_points,
            config['camber_line']['inlet_angle_deg'],
            config['output_settings']['num_points_on_arc']
        )
        # 5. Plot the Airfoil
        if plot_foil:
            ca.plot_airfoil(camber_curve_points, airfoil_top_points, airfoil_bottom_points,
                         config['output_settings']['plot_title'])

        if config['output_settings']['output_airfoil_filename'] != "None":
            ca.write_airfoil_dat(airfoil_top_points, airfoil_bottom_points, camber_curve_points,
                            config['output_settings']['output_airfoil_filename'],
                            config['output_settings']['chord_length_for_export'])

    
    except Exception as e:
        print(f"{e}")
        pass
       
    return airfoil_top_points, airfoil_bottom_points, camber_curve_points

In [ ]:
# Starting with to adjustable params:
# NetworkA -> ChordLength + MaxThickness - using black box function to translate this into our parametrization
config_ = ca.load_config("../blade_design_tools/airfoil_config.yaml")

In [ ]:
config_

# Actor crtitic training
- For the purpose of a prototype, we start modifying a few control points
```python
camber_line:
  control_points_params:
    P1_k_factor: 0.4
    P2_x: 0.5
    P2_y: 0.05
    P3_k_factor: -0.25
  inlet_angle_deg: 45
  outlet_angle_deg: -50
  stagger_angle_deg: -25

top_thickness:
  control_points_params:
    P1_k_factor: 0.2
    P2_x: 0.1
    P2_y: 0.03
    P3_k_factor: 0
  inlet_angle_deg: 40
  outlet_angle_deg: -0
  le_y_thickness: 0.02 # Y-coordinate of P0 for thickness curve (half-thickness at LE)
  p4_y: 0.01           # Y-coordinate of P4 for thickness curve (half-thickness at TE)

bottom_thickness:
  control_points_params:
    P1_k_factor: 0.2
    P2_x: 0.1
    P2_y: 0.03
    P3_k_factor: 0
  inlet_angle_deg: 40
  outlet_angle_deg: -0
  le_y_thickness: 0.02 # Y-coordinate of P0 for thickness curve (half-thickness at LE)
  p4_y: 0.01           # Y-coordinate of P4 for thickness curve (half-thickness at TE)
  ```

In [ ]:
action_idx = {
    1: ["camber_line","control_points_params","P1_k_factor",(0.15,0.5)],
    2: ["camber_line","control_points_params","P3_k_factor",(-0.3,-0.1)],
    3: ["top_thickness","control_points_params","P1_k_factor",(0.15,0.4)],
    4: ["top_thickness","le_y_thickness",(0.01,0.04)]
}

control_params = {
    "top_thickness" : "bottom_thickness"
}

qoi = "pitch2"

def run_script(script_path):
    result = subprocess.run(
        [script_path],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
        shell=True  # only needed if script is not executable
    )
    return result.stdout, result.stderr, result.returncode

def fail_ret(def_path):
    os.chdir(def_path)
    return torch.Tensor([[-1]]), torch.Tensor([[0]])

@torch.no_grad()
def external_eval(actions_, run_id, plot_foil=False):  # action trajectoroy consist of max 4
    actions = actions_.numpy()[0]
    
    config = copy.copy(config_)    
    for step, action_ in enumerate(actions):
        action_data = action_idx[step+1]
        lb = action_data[-1][0]
        ub = action_data[-1][1]

        if len(action_data) == 4:
            config[action_data[0]][action_data[1]][action_data[2]] = lb + action_ * (ub- lb)
    
    
        #for simplicity just consider one thick. param
        if len(action_data) == 3:
            config[action_data[0]][action_data[1]] = lb + action_ * (ub - lb)
            config[control_params[action_data[0]]][action_data[1]] = lb + action_ * (ub - lb)

    config['output_settings']['output_airfoil_filename'] = f"airfoil{run_id}.dat"
    airfoil_top_points, airfoil_bottom_points, _ = generate_shape(config_dict=config, plot_foil=plot_foil)

    #cp test dummy stuff    
    new_f = f"test{run_id}"
    cwd = os.getcwd()
    os.system(f"cp -r uptake/dummy uptake/{new_f}")
    os.system(f"mv airfoil{run_id}.dat uptake/{new_f}/airfoil.dat")
    os.chdir(os.path.join("uptake",new_f))

    stdout, stderr, ret = run_script("../scripts/build.sh")
    if ret != 0:
        print(f"Build script failed with code {ret}")
        return fail_ret(cwd)
        
    stdout, stderr, ret = run_script("../scripts/run_traf.sh")
    if ret != 0:
        print(f"Run script failed with code {ret}")
        return fail_ret(cwd)
    
    if not os.path.exists("fort.40"):
        return fail_ret(cwd)
    
    stdout, stderr, ret = run_script("../scripts/do_post.sh")
    
    #calc loss
    try:
        y,pt = np.genfromtxt(qoi,skip_header=3,usecols=[0,5]).T
        loss = 1-np.trapz(pt,y)    
    except:
        return fail_ret(cwd)
    
    os.chdir(cwd)
    print(f"Run_id: {run_id}")
    
    return torch.Tensor([[1/loss]]), torch.Tensor([[loss]]), run_id

# Refine AC with PPO

In [ ]:
# Networks
class Actor(nn.Module):
    def __init__(self, state_dim, n_numbers, hidden_dim=128):
        super(Actor, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, n_numbers * 2)
        )
        self.n_numbers = n_numbers

    def forward(self, state):
        if state.dim() == 1:
            state = state.unsqueeze(0)
        
        output = self.network(state)
        mean = output[:, :self.n_numbers]
        log_std = output[:, self.n_numbers:]
        log_std = torch.clamp(log_std, -20, 2)
        std = log_std.exp()
        return mean, std

    def get_action_and_log_prob(self, state):
        mean, std = self.forward(state)
        
        dist = distributions.Normal(mean, std)
        action = dist.sample()
        
        log_prob = dist.log_prob(action).sum(dim=-1)
        
        return action, log_prob

    def get_log_prob(self, state, action):
        mean, std = self.forward(state)
        
        if action.dim() != mean.dim():
            action = action.view(mean.shape)
            
        dist = distributions.Normal(mean, std)
        log_prob = dist.log_prob(action).sum(dim=-1)
        return log_prob

    def get_entropy(self, state):
        mean, std = self.forward(state)
        dist = distributions.Normal(mean, std)
        entropy = dist.entropy().sum(dim=-1)
        return entropy

class Critic(nn.Module):
    def __init__(self, state_dim, hidden_dim=128):
        super(Critic, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, state):
        if state.dim() == 1:
            state = state.unsqueeze(0)
        return self.network(state)

class PPOBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.reset()

    def reset(self):
        self.states = []
        self.actions = []
        self.log_probs = []
        self.rewards = []
        self.values = []
        self.dones = []
        self.next_states = []

    def store(self, state, action, log_prob, reward, value, done, next_state):
        self.states.append(state.clone())
        self.actions.append(action.clone())
        self.log_probs.append(log_prob.clone())
        self.rewards.append(reward)
        self.values.append(value.clone())
        self.dones.append(done)
        self.next_states.append(next_state.clone())

    def get(self):
        states = torch.stack(self.states)
        actions = torch.stack(self.actions)
        log_probs = torch.stack(self.log_probs)
        rewards = torch.tensor(self.rewards, dtype=torch.float32)
        values = torch.stack(self.values).squeeze(-1)
        dones = torch.tensor(self.dones, dtype=torch.float32)
        next_states = torch.stack(self.next_states)
        
        return states, actions, log_probs, rewards, values, dones, next_states

    def size(self):
        return len(self.states)

    def clear(self):
        self.reset()


# Simple copy needs to be adjusted 
class qCritic(nn.Module):
    def __init__(self, state_dim, n_qubits=None, n_layers=3):
        super(qCritic, self).__init__()
        
        self.n_qubits = n_qubits if n_qubits is not None else state_dim
        self.input_layer = nn.Linear(state_dim, self.n_qubits)
        self.dev = qml.device("default.qubit", wires=self.n_qubits)
        
        @qml.qnode(self.dev, interface="torch")
        def quantum_circuit(inputs, weights):
            qml.AngleEmbedding(inputs, wires=range(self.n_qubits)) # encode via angle embedding
            
            # Variational quantum circuit with n_layers
            for layer in range(n_layers):
                for qubit in range(self.n_qubits):
                    qml.RX(weights[layer, qubit, 0], wires=qubit)
                    qml.RY(weights[layer, qubit, 1], wires=qubit)
                    qml.RZ(weights[layer, qubit, 2], wires=qubit)
                for qubit in range(self.n_qubits - 1):
                    qml.CNOT(wires=[qubit, qubit + 1])
            
            return [qml.expval(qml.PauliZ(w)) for w in range(self.n_qubits)]
        
        # Initialize quantum weights (n_layers x n_qubits x 3 rotations per qubit)
        weight_shapes = {"weights": (n_layers, self.n_qubits, 3)}
        self.quantum_layer = qml.qnn.TorchLayer(quantum_circuit, weight_shapes)
        
        self.output_layer = nn.Linear(self.n_qubits, 1)
        self.relu = nn.ReLU()
    
    def forward(self, state):
        if state.dim() == 1:
            state = state.unsqueeze(0) # Add batch dimension if needed
        
        x = self.relu(self.input_layer(state))
        x = self.quantum_layer(x)
        value = self.output_layer(x)
        
        return value

In [ ]:
#ppo implementation 
#https://verl.readthedocs.io/en/latest/algo/ppo.html

def test_agent(step, state, action, log_prob, value, external_eval_func, device, state_dim):
        print(f"Step {step}")
        try:
            reward, next_state, _ = external_eval_func(action, step, plot_foil=True)
            
            if next_state.dim() == 2 and next_state.shape[0] == 1:
                next_state = next_state.squeeze(0)
            
            if next_state.shape[0] != state_dim:
                next_state = next_state[:state_dim]
            
            next_state = next_state.to(device)
            
            # Return the transition data instead of storing in buffer
            return {
                'state': state,
                'action': action,
                'log_prob': log_prob,
                'reward': reward.item(),
                'value': value,
                'done': 0.0,
                'next_state': next_state
            }
        
        except Exception as e:
            print(f"Error in external_eval at step {step}: {e}")
            print(f"Action shape: {action.shape}, Action: {action}")
            return None  # Or handle as needed


def compute_gae(rewards, values, next_values, dones, gamma=0.99, lam=0.95): #Generalized Advantage Estimation -> computes adventages values
    advantages = torch.zeros_like(rewards)
    gae = 0
    
    for i in reversed(range(len(rewards))):
        if i == len(rewards) - 1:
            next_value = next_values[-1]
        else:
            next_value = values[i + 1]
            
        delta = rewards[i] + gamma * next_value * (1 - dones[i]) - values[i]
        gae = delta + gamma * lam * (1 - dones[i]) * gae
        advantages[i] = gae
    
    returns = advantages + values
    return advantages, returns

#group policy optimizaion instead of ppo?
def train_ppo(actor, critic,
              external_eval_func, n_numbers=4, state_dim=2, num_episodes=1000, 
              steps_per_episode=100, lr_actor=3e-4, lr_critic=1e-4, gamma=0.99, 
              lam=0.95, clip_ratio=0.2, ppo_epochs=4, batch_size=512, 
              value_loss_coef=0.5, entropy_coef=0.01):
    
    actor_optimizer = optim.Adam(actor.parameters(), lr=lr_actor)
    critic_optimizer = optim.Adam(critic.parameters(), lr=lr_critic)
    
    # State init  - here simple
    state = torch.tensor([0.0], dtype=torch.float32).to(device)
    
    episode_rewards = []
    
    for episode in range(num_episodes):
        print(f"\nEpisode {episode + 1}/{num_episodes}")
        
        buffer = PPOBuffer(steps_per_episode)

        #needs to be revised !    
        print(20*"=")
        
        actions_per_ep = []
        log_prob_per_ep = []
        values_per_ep = []
        ids = [get_id(glock) for _ in range(steps_per_episode)]
        
        for _ in range(steps_per_episode):
            with torch.no_grad():
                action, log_prob = actor.get_action_and_log_prob(state)
                value = critic(state)
            actions_per_ep.append(action)
            log_prob_per_ep.append(log_prob)
            values_per_ep.append(value)
        
        with Pool(processes=steps_per_episode) as pool: # acting state -> pred -> state -> pred
            args = [(step, state, action, log_prob, value, external_eval_func, device, state_dim) 
                    for step,action, log_prob, value in zip(ids, actions_per_ep, log_prob_per_ep,values_per_ep)]
            results = pool.starmap(test_agent, args)
        
            # Sequentially store valid results in buffer
            for result in results:
                if result is not None:
                    buffer.store(**result)
            
        
        if buffer.size() == 0:
            continue
            
        states, actions, old_log_probs, rewards, values, dones, next_states = buffer.get()
        
        index = np.argmax(rewards)
        print(f"Best reward: {actions[index]} - {rewards[index]}")
        #external_eval_func(actions[index], get_id(glock), plot_foil=True)

        
        states = states.to(device)
        actions = actions.to(device) 
        old_log_probs = old_log_probs.to(device)
        rewards = rewards.to(device)
        values = values.to(device)
        dones = dones.to(device)
        next_states = next_states.to(device)
        
        with torch.no_grad():
            next_values = critic(next_states).squeeze(-1)
        
        advantages, returns = compute_gae(rewards, values, next_values, dones, gamma, lam)
        advantages = advantages.to(device)
        returns = returns.to(device)
        
        if advantages.std() > 1e-8:
            advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)
        
        dataset_size = buffer.size()
        
        for ppo_epoch in range(ppo_epochs):
            indices = torch.randperm(dataset_size)
            
            for start_idx in range(0, dataset_size, batch_size):
                end_idx = min(start_idx + batch_size, dataset_size)
                batch_indices = indices[start_idx:end_idx]
                
                batch_states = states[batch_indices]
                batch_actions = actions[batch_indices]
                batch_old_log_probs = old_log_probs[batch_indices]
                batch_advantages = advantages[batch_indices]
                batch_returns = returns[batch_indices]
                
                new_log_probs = actor.get_log_prob(batch_states, batch_actions)
                new_values = critic(batch_states).squeeze(-1)
                
                ratio = torch.exp(new_log_probs - batch_old_log_probs)
                surr1 = ratio * batch_advantages
                surr2 = torch.clamp(ratio, 1 - clip_ratio, 1 + clip_ratio) * batch_advantages
                policy_loss = -torch.min(surr1, surr2).mean()
                
                entropy = actor.get_entropy(batch_states).mean()
                policy_loss = policy_loss - entropy_coef * entropy
                
                value_loss = nn.MSELoss()(new_values, batch_returns)
                
                actor_optimizer.zero_grad()
                policy_loss.backward()
                nn.utils.clip_grad_norm_(actor.parameters(), 0.5)
                actor_optimizer.step()
                
                critic_optimizer.zero_grad()
                value_loss.backward()
                nn.utils.clip_grad_norm_(critic.parameters(), 0.5)
                critic_optimizer.step()
        
        # Log episode results
        episode_reward = rewards.median().item()
        episode_rewards.append(episode_reward)
        if episode>0 and episode%10==0:
            print(f"Episode {episode + 1} - Total Reward: {episode_reward:.4f}")
            print(f"  Mean Reward: {rewards.mean().item():.4f}")
            print(f"  Policy Loss: {policy_loss.item():.4f}")
            print(f"  Value Loss: {value_loss.item():.4f}")
            print(f"  Entropy: {entropy.item():.4f}")
        
        # Print running statistics
        if len(episode_rewards) >= 10:
            recent_mean = np.mean(episode_rewards[-10:])
            print(f"  Recent 10 episodes average: {recent_mean:.4f}")
    
    return actor, critic, episode_rewards

In [ ]:
#needs to have a larger number of 
n_numbers=4 
state_dim=1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
    
# Initialize networks
actor = Actor(state_dim, n_numbers).to(device)
critic = Critic(state_dim).to(device)


actor, critic, rewards = train_ppo(actor, critic,
            external_eval_func=external_eval,  
            n_numbers=1,
            num_episodes=50,  
            steps_per_episode=4  
        )
    
print(f"\nTraining completed!")
print(f"Final rewards: {rewards[-5:]}") 

In [ ]:
plt.plot(rewards)
plt.ylabel("Average Reward")
plt.xlabel("Iteration")

In [ ]:
#needs to have a larger number of 
n_numbers=4 
state_dim=1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
    
# Initialize networks
actor = Actor(state_dim, n_numbers).to(device)
critic = qCritic(state_dim).to(device)


actor, critic, rewards = train_ppo(actor, critic,
            external_eval_func=external_eval,  
            n_numbers=1,
            num_episodes=50,  
            steps_per_episode=4  
        )
    
print(f"\nTraining completed!")
print(f"Final rewards: {rewards[-5:]}") 

In [ ]:
plt.plot(rewards)
plt.ylabel("Average Reward")
plt.xlabel("Iteration")

## Result:
- Baseline loss: 0.044501
- Best loss s_ppo: 0.038491
- Best loss q_ppo: 0.038025

In [ ]:
actor, critic, rewards2 = train_ppo(actor, critic,
            external_eval_func=external_eval,  
            n_numbers=1,
            num_episodes=100,  
            steps_per_episode=16  
        )

In [ ]:
def external_eval_no(actions, run_id, plot_foil=True):
    actions_str = json.dumps(actions.tolist())    
    plot_foil_str = 'true' if plot_foil else 'false'
    
process = subprocess.Popen(
        ['python', 'uptake/external_eval.py', actions_str, str(run_id), plot_foil_str],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    stdout, stderr = process.communicate()
    print(stdout)
    if process.returncode != 0:
        print(f"Subprocess failed: {stderr}")
        return 9999
    
    try:
        return json.loads(stdout.strip())
    except json.JSONDecodeError:
        print(f"Invalid output: {stdout}")
        return 9999

In [ ]:

"""
    1: ["camber_line","control_points_params","P1_k_factor",],
    2: ["camber_line","control_points_params","P3_k_factor",(-0.3,-0.1)],
    3: ["top_thickness","control_points_params","P1_k_factor",(0.15,0.4)],
    4: ["top_thickness","le_y_thickness",(0.01,0.04)]
"""


err_log_name = 'coeff_test_1.csv'
seed = 1
cpu_count = 30

np.random.seed(seed)

counter = Value('i', 0)
counter_lock = Lock()


def to_log(elem, uuid, err):
    with counter_lock:
        print(elem[0])
        try:
            log = {
                "ID": uuid,
                "1": f"{elem[0]:.5f}" ,
                "2": f"{elem[1]:.5f}",
                "3": f"{elem[2]:.5f}",
                "4": f"{elem[3]:.5f}",
                "Error": err
            }
            pd.DataFrame(log, index=[0]).to_csv(err_log_name, header=not os.path.exists(err_log_name), index=False, mode='a')
        except Exception as e:
            print(f"Ex: {e}")
        print("Save done")

def create_uid_():
    with counter_lock:
        ret_val = '0' * (5 - len(str(counter.value))) + str(counter.value)
        counter.value += 1
    return ret_val

def evaluate(individual):
    uuid_ = create_uid_()
    err = external_eval_no(individual,uuid_)
    to_log(individual, uuid_, err)
    return err, uuid_

def objective_function(x):
    result, uuid = evaluate(x)
    str_list = ";".join([f"{elem:.4f}" for elem in x])
    os.system(f"echo '{uuid};{result};{str_list}'>>log_file.csv")
    return result

# Define bounds for the parameters
bounds = [(0.1,0.5), (-0.4,-0.1), (0.15,0.5), (0.01,0.05)]  # Example bounds

result = differential_evolution(objective_function, bounds, strategy='best1bin', maxiter=100, popsize=30, tol=0.01, mutation=(0.5, 1), recombination=0.5, workers=cpu_count)

print("Best cost:", result.fun)
print("Best position:", result.x)
